### описание


В этом проекте мы строим прототип рекомендательной системы блюд в ресторане.
1.Мы хотим чтобы клиенту подбирали блюдо, аналогичное по ингридиентам. 

2.Мы хотим сделать фильтр, чтобы если человек не есть соль, не рекомендовать ему блюда в который есть соль.

3.Мы хотим в этом фильтре более широкую фильтрацию, например если человек не есть мясо, не рекомендовать ему мясо.




Проект делается на предобученнйо модели от facebookа на руссуом языке.



### Рабочий датасет
#### импорты

In [1]:
from gensim.models import KeyedVectors
import numpy as np
from scipy.spatial.distance import cosine
import pandas as pd

Настройки и глобальные переменные

In [2]:
pd.set_option('display.max_colwidth', None)


#### функции для работы

In [3]:
def recipe_vector(recipe, model):
    vectors = [model[word] for word in recipe if word in model.key_to_index]
    return np.mean(vectors, axis=0)

def recipe_similarity(recipe1, recipe2, model):
    vec1 = recipe_vector(recipe1, model)
    vec2 = recipe_vector(recipe2, model)
    return 1 - cosine(vec1, vec2)  # cosine returns distance, so we subtract it from 1 to get similarity


def find_similar(name, df, model):
    if name not in df['meal'].values:
        return f"{name} not found in the recipes list."
    
    # Get the vector for the given recipe
    given_vector = df[df['meal'] == name]['vector'].values[0]
    
    # Calculate the similarity between the given recipe and all others
    df['similarity'] = df['vector'].apply(lambda x: 1 - cosine(given_vector, x))

    # Return the sorted DataFrame, excluding the given recipe
    return df[df['meal'] != name].sort_values('similarity', ascending=False)[['meal', 'similarity']]



def calculate_similarities(df):

    similarities = {}
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            recipe1 = df.iloc[i]['meal']
            recipe2 = df.iloc[j]['meal']
            vec1 = df.iloc[i]['vector']
            vec2 = df.iloc[j]['vector']
            similarity = 1 - cosine(vec1, vec2)  # calculate similarity
            similarities[(recipe1, recipe2)] = similarity
            
    return similarities



#### загрузка предобученной модели

In [4]:

# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format('/home/nikita/Kirill_restaraunt/cc.ru.300.vec', limit=1000000)



# Compute cosine similarity


проверка на адекватность

In [5]:
# See the word vector
vector = model.get_vector('привет')

In [6]:
similarity = model.similarity('привет', 'здравствуйте')
print(similarity)

similar_words = model.most_similar('привет', topn=20)

# Print each of the similar words along with their similarity scores
for word, similarity in similar_words:
    print(f"{word}: {similarity}")

0.67381525
приветик: 0.7990784049034119
Привет: 0.7965432405471802
приветики: 0.7429574131965637
привет.: 0.715659499168396
ПРивет: 0.7086106538772583
привет-привет: 0.6934460401535034
здравствуйте: 0.6738153100013733
приветы: 0.6671836972236633
ПРИВЕТ: 0.6660736799240112
Приветы: 0.6552184820175171
здрасти: 0.640758752822876
здрасте: 0.6292753219604492
Приветики: 0.6285543441772461
Приветик: 0.6160926818847656
Привет.: 0.6125413179397583
Привет-привет: 0.6053478717803955
здрасьте: 0.5969064235687256
здарова: 0.5928993225097656
Здрасте: 0.5867396593093872
привет.у: 0.5848491787910461


In [7]:
similar_words = model.most_similar('мясо', topn=20)
similar_words

[('мяско', 0.7551317811012268),
 ('мясо.', 0.7505342960357666),
 ('Мясо', 0.7483969926834106),
 ('говядину', 0.7157896757125854),
 ('свинину', 0.7041599750518799),
 ('сало', 0.6996752619743347),
 ('курятину', 0.6891534924507141),
 ('мяса', 0.6782190799713135),
 ('мясцо', 0.662449836730957),
 ('молоко', 0.6586489677429199),
 ('курицу', 0.657137930393219),
 ('крольчатину', 0.6550291180610657),
 ('маринованное', 0.6516428589820862),
 ('жареное', 0.6498854756355286),
 ('телятину', 0.6440608501434326),
 ('свинина', 0.6413251757621765),
 ('поджаренное', 0.6401798129081726),
 ('жаренное', 0.6380771398544312),
 ('говядина', 0.6363006830215454),
 ('прожаренное', 0.6309633255004883)]

In [8]:
similar_words = model.most_similar('шаурма', topn=20)

# Print each of the similar words along with their similarity scores
for word, similarity in similar_words:
    print(f"{word}: {similarity}")

шаверма: 0.7137317657470703
Шаурма: 0.6821454763412476
шаурму: 0.6748165488243103
пицца: 0.6270208954811096
шавермы: 0.6152158379554749
шаурмы: 0.6122849583625793
Шаверма: 0.6072588562965393
шаверму: 0.5909028649330139
шаурме: 0.5860306024551392
шаурмой: 0.5694023966789246
чебурек: 0.5655691027641296
кебаб: 0.5590173602104187
самса: 0.5557743906974792
чебуреки: 0.5554170608520508
шашлычная: 0.5545408129692078
бастурма: 0.5535683631896973
колбаса: 0.5519727468490601
забегаловка: 0.5424509048461914
Чебурек: 0.5356056690216064
котлета: 0.5320447683334351


In [9]:
similarity = model.similarity('курица', 'банан')
print(similarity)


0.33091888


Зададим пару блюд и попробуем найти похожесть

In [10]:
# Ingredients of 'греческий салат'
greek_salad_ingredients = ['Помидоры', 'Огурцы', 'Перец сладкий', 'Лук репчатый салатный', 'Сыр фета', 'Маслины', 'Лимон', 'Масло оливковое', 'Базилик свежий', 'Соль', 'Салат', 'Перец молотый']

# Ingredients of 'салат цезарь'
caesar_salad_ingredients = ['курица (филе)', 'сыр твёрдый', 'батон белый', 'салат зеленый', 'помидоры черри', 'Яйца', 'ядра грецких орехов', 'масло оливковое', 'Чеснок', 'Горчица', 'сок лимона', 'Соль', 'Перец']

coffee_ingridients=['кофе', 'молоко', 'корица']

cappuchino_ingridients=['кофе', 'вода', 'молоко']

dummy_ingridients=['молоко молотый','хрен тертый','говядина','сахар']



In [11]:

# Example usage:

# Compare Greek salad and Caesar salad
print(recipe_similarity(greek_salad_ingredients, caesar_salad_ingredients, model))
print(recipe_similarity(greek_salad_ingredients, dummy_ingridients, model))
print(recipe_similarity(greek_salad_ingredients, coffee_ingridients, model))
print(recipe_similarity(cappuchino_ingridients, coffee_ingridients, model))

0.8340020179748535
0.42332378029823303
0.3317553699016571
0.8857256770133972


Создандим пробный датасет с блюдами

In [19]:
import pandas as pd

data = {
    "meal": ["Капучино", "Капучино с корицей", "Латте", "Раф", "Американо", "Греческий салат", "Салат цезарь с курицей", "Салат цезарь с креветками", "Борщ", "Шаурма с курицей", "Бургер классический","Фейковая чушь"],
    "category": ["кофе", "кофе", "кофе", "кофе", "кофе", "салаты", "салаты", "салаты", "супы", "Фастфуд", "Фастфуд", "Фастфуд"],
    "ingredients": [
        ["кофе молотый", "вода", "молоко"],
        ["кофе", "молоко", "корица"],
        ["кофе молотый", "вода", "молоко"],
        ["Вода","сливки", "кофе молотый"],
        ["кофе", "молотый"],
        ["Помидоры","Огурцы","Перец сладкий","Лук репчатый салатный","Сыр фета","Маслины","Лимон","Масло оливковое","Базилик свежий", "Соль","Салат" , "Перец молотый"],
        ["курица (филе)" , "сыр твёрдый" , "батон белый" , "салат зеленый" , "помидоры черри" , "Яйца", "ядра грецких орехов", "масло оливковое", "Чеснок", "Горчица", "сок лимона" , "Соль", "Перец"],
        ["Креветки", "сыр твёрдый" , "батон белый" , "салат зеленый" , "помидоры черри" , "Яйца", "ядра грецких орехов", "масло оливковое", "Чеснок", "Горчица", "сок лимона" , "Соль", "Перец"],
        ["Мясо свиное","Картофель","Лук репчатый" , "Свекла" , "Морковь" , "Капуста" , "Томатная паста", "Масло растительное", "Зелень" , "Сок лимона" , "Сметана" , "Лавровый лист" , "Чеснок" , "Соль", "сахар", "перец"],
        ["Куриное филе","Огурцы","Помидоры","Капуста","Майонез","Кетчуп","Лаваш тонкий","Растительное масло","Соль"],
        ["Булочки для бургера","Фарш мясной","Сыр твердый","Помидор" ,"Огурец соленый" ,"Лук синий","Салатные листья","Майонез" ,"Кетчуп" ,"Перец черный" ,"Соль"],
        ["молоко", "конский хвост","тертый хрен", "пиво", "сыр","прокисший кефир"]
    ]
}

data = pd.DataFrame(data)
data['vector'] = data['ingredients'].apply(lambda x: recipe_vector(x, model))
display(data.head(5))


,meal,category,ingredients,vector
0,Капучино,кофе,"[кофе молотый, вода, молоко]","[0.0798, 0.10675, -0.1189, 0.01865, 0.0193, -0.07915, 0.0089, -0.0561, -0.0568, -0.04625, -0.00925, 0.02, 0.0404, 0.017, 0.00465, 0.053449996, -0.031850003, -0.02515, -0.0328, -0.10335, -0.0014500001, -0.00089999987, -0.056649998, -0.07955, 0.0225, 0.00815, -0.06505, -0.013300001, 0.009850001, 0.03725, 0.031999998, 0.0228, 0.0279, -0.06175, -0.0255, -0.03275, 0.0447, -0.1429, -0.054900005, -0.15715, 0.08145, -0.070999995, 0.011499999, 0.0013000001, 0.077649996, 0.01, 0.0975, 0.0819, 0.07665, 0.00045000017, 0.08385, 0.025249999, -0.081949994, -0.00015000068, -0.047300003, 0.08095, -0.109, -0.03195, -0.0072000003, 0.0519, -0.0323, 0.00835, 0.0592, -0.014250001, 0.0131, -0.02315, -0.0183, 0.0625, 0.0397, 0.054250002, 0.0301, -0.03515, 0.041249998, 0.02245, -0.0161, 0.05555, -0.100899994, 0.0143, -0.0338, 0.02325, 0.0125, 0.0497, -0.0849, -0.05755, -0.046, -0.002, -0.17565, 0.0458, 0.0196, 0.06945, 0.0053499993, 0.004900001, 0.102800004, 0.063549995, -0.04425, -0.069, -0.052550003, 0.018199999, -0.02725, 0.0479, ...]"
1,Капучино с корицей,кофе,"[кофе, молоко, корица]","[0.113966666, 0.0173, -0.02393333, 0.0011, 0.10073334, -0.0995, 0.082333334, 0.123399995, -0.07503333, -0.05963333, -0.0263, 0.046833333, -0.018, 0.064733334, -0.0804, -0.040833335, -0.050033335, -0.062233333, -0.08923334, -0.012100001, 0.0027666662, 0.010733333, -0.0079666665, -0.024566667, 0.054333333, -0.056866664, -0.021166667, -0.036533333, 0.104300015, 0.008066666, -0.0423, -0.040166665, 0.058666665, 0.062333334, -0.010266667, -0.021066666, 0.021866666, -0.09553333, -0.033666667, -0.114133336, 0.067899995, -0.022466667, 0.06776667, 0.0074666664, 0.049133334, 0.073633336, 0.11373333, 0.071433336, 0.116066664, 0.0328, 0.11436667, 0.058366667, -0.06956666, -0.032233335, 0.021800002, 0.05226667, -0.050366666, -0.035800003, -0.00013333373, -0.066199996, -0.027933335, -0.01, 0.0232, 0.022200001, 0.005866667, 0.0054666675, -0.08523333, 0.060200002, -0.06423333, 0.0815, -0.05773334, -0.06943333, 0.02363333, -0.0371, -0.023500001, 0.06736667, -0.08166667, 0.06143333, -0.013700001, 0.0029333339, 0.06776667, 0.059266668, -0.012966666, -0.05236667, -0.0030333363, 0.027066665, -0.13563333, 0.008700001, -0.018733334, -0.0117666675, 0.0238, 0.06713333, 0.0239, -0.008333335, -0.042266667, -0.004633332, -0.026633332, 0.054033328, -0.027633334, 0.0529, ...]"
2,Латте,кофе,"[кофе молотый, вода, молоко]","[0.0798, 0.10675, -0.1189, 0.01865, 0.0193, -0.07915, 0.0089, -0.0561, -0.0568, -0.04625, -0.00925, 0.02, 0.0404, 0.017, 0.00465, 0.053449996, -0.031850003, -0.02515, -0.0328, -0.10335, -0.0014500001, -0.00089999987, -0.056649998, -0.07955, 0.0225, 0.00815, -0.06505, -0.013300001, 0.009850001, 0.03725, 0.031999998, 0.0228, 0.0279, -0.06175, -0.0255, -0.03275, 0.0447, -0.1429, -0.054900005, -0.15715, 0.08145, -0.070999995, 0.011499999, 0.0013000001, 0.077649996, 0.01, 0.0975, 0.0819, 0.07665, 0.00045000017, 0.08385, 0.025249999, -0.081949994, -0.00015000068, -0.047300003, 0.08095, -0.109, -0.03195, -0.0072000003, 0.0519, -0.0323, 0.00835, 0.0592, -0.014250001, 0.0131, -0.02315, -0.0183, 0.0625, 0.0397, 0.054250002, 0.0301, -0.03515, 0.041249998, 0.02245, -0.0161, 0.05555, -0.100899994, 0.0143, -0.0338, 0.02325, 0.0125, 0.0497, -0.0849, -0.05755, -0.046, -0.002, -0.17565, 0.0458, 0.0196, 0.06945, 0.0053499993, 0.004900001, 0.102800004, 0.063549995, -0.04425, -0.069, -0.052550003, 0.018199999, -0.02725, 0.0479, ...]"
3,Раф,кофе,"[Вода, сливки, кофе молотый]","[0.0040000007, 0.0782, -0.0856, 0.05025, 0.037600003, -0.06905, 0.05495, -0.0413, 0.012650001, -0.02465, -0.0136, 0.013800001, 0.028, -0.058149997, 0.0041499995, 0.03145, -0.0174, -0.0563, -0.0024500005, -0.084199995, 0.0394, -0.0314, -0.121649995, -0.06455, 0.025799999, 0.019749999, 0.0392, 0.05195, -0.033, -0.033299997, -0.00715, 0.027049996, 0.011149999, -0.054199997, 0.03925, -0.08005, -0.014, -0.1618, 0.047750004, -0.12245, 0.0632, 0.0526, 0.

In [20]:
#посмотрим ближайшие блюда на салат цезарь с креветками
similar_recipes = find_similar('Салат цезарь с креветками', data, model)
print(similar_recipes)


                      meal  similarity
6   Салат цезарь с курицей    0.993183
8                     Борщ    0.877540
9         Шаурма с курицей    0.870013
5          Греческий салат    0.852914
10     Бургер классический    0.840598
3                      Раф    0.550698
1       Капучино с корицей    0.367964
4                Американо    0.346486
11           Фейковая чушь    0.341572
0                 Капучино    0.339755
2                    Латте    0.339755


In [21]:
#посмотрим ближайшие блюда на салат цезарь с креветками
similar_recipes = find_similar('Фейковая чушь', data, model)
print(similar_recipes)


                         meal  similarity
1          Капучино с корицей    0.678963
0                    Капучино    0.621774
2                       Латте    0.621774
4                   Американо    0.536839
8                        Борщ    0.446271
3                         Раф    0.438263
10        Бургер классический    0.397510
9            Шаурма с курицей    0.392336
5             Греческий салат    0.343106
7   Салат цезарь с креветками    0.341572
6      Салат цезарь с курицей    0.331729


In [22]:

# Calculate recipe vectors
df['vector'] = df['ingridients'].apply(lambda x: recipe_vector(x, model))

# Calculate similarities between all pairs of recipes
similarities = {}
for i in range(len(df)):
    for j in range(i+1, len(df)):
        recipe1 = df.iloc[i]['meal']
        recipe2 = df.iloc[j]['meal']
        vec1 = df.iloc[i]['vector']
        vec2 = df.iloc[j]['vector']
        similarity = 1 - cosine(vec1, vec2)  # calculate similarity
        similarities[(recipe1, recipe2)] = similarity

# Find the pair of recipes with the highest similarity
most_similar_recipes = max(similarities, key=similarities.get)
print(f"The most similar recipes are {most_similar_recipes[0]} and {most_similar_recipes[1]} with a similarity of {similarities[most_similar_recipes]}")




# Example usage:
similar_recipes = find_similar('Салат цезарь с креветками', df, model)
display(similar_recipes)


The most similar recipes are Капучино and Латте with a similarity of 1


,meal,similarity
6,Салат цезарь с курицей,0.982280
8,Борщ,0.877540
5,Греческий салат,0.852914
10,Бургер классический,0.840598
9,Шаурма с курицей,0.833169
3,Раф,0.550698
1,Капучино с корицей,0.367964
4,Американо,0.346486
11,Молочко,0.341572
0,Капучино,0.339755


In [23]:
# Example usage:
similar_recipes = find_similar('Борщ', df, model)
print(similar_recipes)


                         meal  similarity
6      Салат цезарь с курицей    0.888481
9            Шаурма с курицей    0.879114
7   Салат цезарь с креветками    0.877540
5             Греческий салат    0.837807
10        Бургер классический    0.821697
3                         Раф    0.600030
1          Капучино с корицей    0.463218
0                    Капучино    0.458460
2                       Латте    0.458460
11                    Молочко    0.446271
4                   Американо    0.410575


##### датасет с примерным меню из ресторана.

In [32]:

data = {
    'meal': [
        'Омлет с грибами', 'Омлет с сыром', 'Гречневая каша', 'Овсяная каша', 'Борщ', 'Солянка',
        'Тирамису', 'Чизкейк Ньйю-Йорк классический', 'Цезарь', 'Греческий', 'Стейк из говядины', 'Стейк из свинины',
        'Лосось на гриле', 'Тунец в соусе терияки', 'Эспрессо', 'Капучино', 'Кофе на совевом молоке',
        'Зеленый чай', 'Черный чай','Шаурма с курицей','Бургер классический','Фейковое блюдо'
    ],
    'category': [
        'Омлеты', 'Омлеты', 'Каша', 'Каша', 'Супы', 'Супы', 'Десерты', 'Десерты', 'Салаты',
        'Салаты', 'Стейк', 'Стейк', 'Рыба', 'Рыба', 'Кофе', 'Кофе', 'Кофе', 'Чай', 'Чай','Фастфуд','Фастфуд','Фастфуд'
    ],
    'meal_type': [
        'Завтраки', '', '', '', '', '', '', '', '', '', 'Основные блюда', 'Основные блюда',
        'Основные блюда', 'Основные блюда', 'Напитки', 'Напитки', 'Напитки', 'Напитки', 'Напитки','Основные блюда','Основные блюда','Основные блюда'
    ],
    'ingredients': [
        ['яйца', 'шампиньоны', 'масло сливочное', 'соль', 'перец чёрный'], ['яйца','сыр любой','растительное масло','соль'], ['гречка','вода','растительное масло','соль'], ['овсяные хлопья','молоко','сливочное масло','бананы','сахар','соль'], ['курица','картошка','капуста','лук','морковь','помидоры','свекла','вода','растительное масло','соль']
        , ['ветчина','копченая колбаса','оливки','бульон говяжий','лук','томатная паста','растительное масло','сметана'], ['печенье савоярди','маскарпоне','сахар','яйца','кофе','какао'], ['печенье песочное','сливочное масло','мускатный орех','сливочный сыр','сахар','сливки жирные','яйца','лимонный сок','ванильный экстракт'], ["курица (филе)" , "сыр твёрдый" , "батон белый" , "салат зеленый" , "помидоры черри" , "Яйца", "ядра грецких орехов", "масло оливковое", "Чеснок", "Горчица", "сок лимона" , "Соль", "Перец"],  ["Помидоры","Огурцы","Перец сладкий","Лук репчатый салатный","Сыр фета","Маслины","Лимон","Масло оливковое","Базилик свежий", "Соль","Салат" , "Перец молотый"],['вырезка говяжья', 'вырезка телячья', 'масло растительное', 'соль', 'перец чёрный молотый',
        'масло сливочное']
, ['Свиная корейка без кости','Соевый соус','Вода','Чеснок','перец'],
        ['филе лосося','cоль','перец','соевый соус'], ['тунец стейк','Имбирь','cоевый соус','Сахар','чеснок','уксус'], ['кофе','вода'], ['кофе','молоко','вода'], ['кофе','соевое молоко','вода'], ['чай зеленый','вода'], ['чай черный','вода']
    ,["Куриное филе","Огурцы","Помидоры","Капуста","майонез","кетчуп","Лаваш тонкий","Растительное масло","Соль"],
           ["Булочки для бургера","Фарш мясной","Сыр твердый","Помидор" ,"Огурец соленый" ,"Лук синий","Салатные листья","Майонез" ,"Кетчуп" ,"Перец черный" ,"Соль"],  ["молоко", "конский хвост","тертый хрен", "пиво", "сыр","прокисший кефир"]
    ]
    
}

df = pd.DataFrame(data)
# Calculate recipe vectors
df['vector'] = df['ingredients'].apply(lambda x: recipe_vector(x, model))

display(df.head())


,meal,category,meal_type,ingredients,vector
0,Омлет с грибами,Омлеты,Завтраки,"[яйца, шампиньоны, масло сливочное, соль, перец чёрный]","[0.0982, 0.050266664, -0.029566666, 0.1077, 0.10763333, 0.023366665, 0.047933336, -0.0128666675, -0.03186667, -0.025266664, -0.035633333, 0.013900001, -0.03743333, -0.0411, 0.056033332, 0.017433332, 0.003033335, -0.08276667, -0.14013334, -0.018733332, 0.009866666, 0.052666668, -0.0021333322, -0.082666665, 0.09516666, -0.018266667, 0.0622, -0.0139999995, 0.03066667, 0.044999998, 0.022166664, 0.036033332, 0.061566666, 0.087233335, 0.005633333, 0.0027333337, -0.020200001, -0.0977, -0.017833335, -0.06756666, 0.13463333, 0.0018666672, -0.045066666, -0.037499998, 0.042699996, -0.0432, 0.039199997, 0.0776, 0.029166667, 0.08089999, 0.034133334, 0.022733333, -0.0026333334, 0.05, -0.08616667, 0.07943333, -0.0072333333, -0.045033336, 0.05243333, -0.008833334, -0.023333333, 0.0016666666, 0.013533334, 0.040633332, -0.036233336, 0.06446666, 0.07326666, 0.005633333, -0.04193333, 0.0862, -0.009533331, -0.028466666, -0.07676667, 0.019833334, -0.026466668, 0.1276, -0.0675, 0.013333333, 0.0113, 0.05736667, 0.011866666, 0.0504, -0.033833336, -0.10926667, 0.0032, 0.028766667, -0.112500004, 0.051066667, -0.06643333, 0.032733332, 0.0006000002, 0.040900003, -0.0010000002, 0.013599996, 0.0581, -0.022266665, 0.0188, -0.040566664, -0.047533333, 0.113900006, ...]"
1,Омлет с сыром,Омлеты,,"[яйца, сыр любой, растительное масло, соль]","[0.1587, 0.07224999, -0.043349996, 0.118599996, 0.1276, 0.0414, 0.060850002, -0.0198, -0.061050005, -0.0503, -0.03985, 0.025400002, -0.06585, -0.026600001, 0.08405, 0.008949999, -0.04155, -0.115, -0.16505, -0.02615, 0.00845, 0.0686, 0.00090000033, -0.12035, 0.14535001, -0.014800001, 0.079349995, -0.023999998, 0.04, 0.0447, 0.039399996, 0.04285, 0.086449996, 0.1328, -0.0045999996, 0.0036500003, -0.02265, -0.1272, -0.031650003, -0.1014, 0.1664, 0.011700001, -0.0463, -0.04795, 0.0546, -0.07395, 0.052, 0.114199996, 0.03005, 0.10875, 0.06415, 0.0098, -0.00025000004, 0.07195, -0.0963, 0.09415, 0.00055, -0.040900003, 0.0846, 0.0142, -0.02245, -0.0011, 0.0306, 0.05105, -0.058300003, 0.065799996, 0.0794, 0.009199999, -0.04065, 0.0819, 0.0037500039, -0.0159, -0.09865, 0.02235, -0.03145, 0.1911, -0.080800004, -0.0067999996, 0.0489, 0.1016, 0.0143, 0.06125, -0.049950004, -0.14570001, -0.0008500004, 0.02925, -0.1506, 0.06635, -0.08585, 0.02605, -0.01245, 0.043050002, -0.0112000005, 0.024099998, 0.0723, -0.02775, 0.03165, -0.04175, -0.06665, 0.15540001, ...]"
2,Гречневая каша,Каша,,"[гречка, вода, растительное масло, соль]","[0.08013334, 0.052733332, -0.0484, 0.040566664, 0.044766665, -0.08036666, 0.0329, -0.07553334, -0.085666664, -0.044999998, 0.008166666, 0.046266664, 0.0062666647, -0.0331, -0.004733333, 0.091699995, -0.021633334, -0.057533335, -0.11803333, -0.0476, 0.0115, 0.028066665, -0.063766666, -0.078933336, 0.007866666, 0.015666667, 0.030699998, 0.011066668, 0.022166668, 0.007866667, 0.05443333, 0.053833336, 0.036333334, 0.007033333, -0.0193, -0.040666666, 0.0112999985, -0.14583333, -0.12353334, -0.106599994, 0.12936668, -0.0429, -0.0069, -0.0103, 0.040766668, 0.0032333334, 0.07153333, 0.0455, 0.094399996, 0.017833332, 0.0959, 0.004333333, -0.054199997, 0.043500002, -0.08646667, 0.1423, -0.0753, 0.022266665, 0.0114, 0.034466665, -0.030099997, 0.0028666668, 0.09573334, 6.666655e-05, -0.096066676, -0.002666667, 0.048366666, 0.027666666, -0.025633333, 0.05943333, 0.086666666, -0.014633334, -0.045933336, 0.050333332, -0.017666668, 0.1525, -0.079733334, 0.009, -0.0148, 0.054433335, -0.005233334, 0.061433334, -0.098966666, -0.08653333, 0.004766667, 0.013700001, -0.13363333, 0.03223333, 0.0115, 0.0639, -0.0186, 0.064733334, 0.046000004, 0.09433333, -0.0036666666, -0.05646667, -0.028800001, 0.04186667, -0.0193, 0.10016667, ...]"
3,Овсяная каша,Каша,,"[овсяные хлопья, молоко, сливочное масло, бананы, сахар, соль]","[0.135875, 0.024825001, -0.0414, 0.0512, 0.092075005, -0.04685, 0.0279, 0.015725002, -0.0

In [33]:

# Calculate similarities between all pairs of recipes
similarities = {}
for i in range(len(df)):
    for j in range(i+1, len(df)):
        recipe1 = df.iloc[i]['meal']
        recipe2 = df.iloc[j]['meal']
        vec1 = df.iloc[i]['vector']
        vec2 = df.iloc[j]['vector']
        similarity = 1 - cosine(vec1, vec2)  # calculate similarity
        similarities[(recipe1, recipe2)] = similarity

# Find the pair of recipes with the highest similarity
most_similar_recipes = max(similarities, key=similarities.get)
print(f"The most similar recipes are {most_similar_recipes[0]} and {most_similar_recipes[1]} with a similarity of {similarities[most_similar_recipes]}")


The most similar recipes are Эспрессо and Кофе на совевом молоке with a similarity of 1


Посмотрим схожести:

In [34]:
similar_recipes = find_similar('Бургер классический', df, model)
similar_recipes

,meal,similarity
19,Шаурма с курицей,0.883737
8,Цезарь,0.840208
9,Греческий,0.838064
11,Стейк из свинины,0.711678
13,Тунец в соусе терияки,0.676582
12,Лосось на гриле,0.542954
10,Стейк из говядины,0.531792
4,Борщ,0.524275
3,Овсяная каша,0.508904
0,Омлет с грибами,0.508350


In [28]:
similar_recipes = find_similar('Омлет с сыром', df, model)
similar_recipes

,meal,similarity
0,Омлет с грибами,0.982504
7,Чизкейк Ньйю-Йорк классический,0.854628
10,Стейк из говядины,0.806151
3,Овсяная каша,0.803035
2,Гречневая каша,0.712931
4,Борщ,0.686090
6,Тирамису,0.677403
19,Шаурма с курицей,0.595181
5,Солянка,0.576468
8,Цезарь,0.574110


In [29]:
find_similar('Овсяная каша',df,model)

,meal,similarity
0,Омлет с грибами,0.808263
1,Омлет с сыром,0.803035
2,Гречневая каша,0.794545
10,Стейк из говядины,0.792733
7,Чизкейк Ньйю-Йорк классический,0.786477
6,Тирамису,0.774347
4,Борщ,0.754690
15,Капучино,0.737961
13,Тунец в соусе терияки,0.706252
5,Солянка,0.644158


,meal,similarity
14,Эспрессо,0.965914
16,Кофе на совевом молоке,0.965914
6,Тирамису,0.773755
18,Черный чай,0.753080
17,Зеленый чай,0.753080
3,Овсяная каша,0.737961
2,Гречневая каша,0.731280
4,Борщ,0.633163
11,Стейк из свинины,0.577937
7,Чизкейк Ньйю-Йорк классический,0.563445


In [35]:

find_similar('Фейковое блюдо',df,model)

,meal,similarity
15,Капучино,0.693725
3,Овсяная каша,0.687393
6,Тирамису,0.678631
5,Солянка,0.651541
7,Чизкейк Ньйю-Йорк классический,0.584493
4,Борщ,0.579570
16,Кофе на совевом молоке,0.571851
14,Эспрессо,0.571851
0,Омлет с грибами,0.555873
13,Тунец в соусе терияки,0.530153


In [38]:


def find_similar_rec(recipe_name, df, model):
    """function for interpritaion of meals similarities"""
    target_recipe = df[df['meal'] == recipe_name]
    target_vector = target_recipe['vector'].values[0]
    similarities = []

    for i, row in df.iterrows():
        if row['meal'] != recipe_name:
            recipe_vector = row['vector']
            similarity = 1 - cosine(target_vector, recipe_vector)
            ingredient_similarities = []
            for ingredient in row['ingredients']:
                if ingredient in model:
                    ingredient_vector = model[ingredient]
                    ingredient_similarity = 1 - cosine(target_vector, ingredient_vector)
                    ingredient_similarities.append((ingredient, ingredient_similarity))
            similarities.append((row['meal'], similarity, ingredient_similarities))

    return sorted(similarities, key=lambda x: x[1], reverse=True)


In [37]:
find_similar_rec('Цезарь',df,model)

[('Бургер классический',
  0.840208113193512,
  [('Помидор', 0.5887152552604675),
   ('Майонез', 0.5891571044921875),
   ('Кетчуп', 0.4914434552192688),
   ('Соль', 0.7715861201286316)]),
 ('Греческий',
  0.8340020179748535,
  [('Помидоры', 0.6314594149589539),
   ('Огурцы', 0.5720213055610657),
   ('Маслины', 0.46664586663246155),
   ('Лимон', 0.5986625552177429),
   ('Соль', 0.7715861201286316),
   ('Салат', 0.4430389404296875)]),
 ('Шаурма с курицей',
  0.8241896629333496,
  [('Огурцы', 0.5720213055610657),
   ('Помидоры', 0.6314594149589539),
   ('Капуста', 0.5874060988426208),
   ('майонез', 0.40263858437538147),
   ('кетчуп', 0.3417719304561615),
   ('Соль', 0.7715861201286316)]),
 ('Стейк из свинины',
  0.7854945659637451,
  [('Вода', 0.47554388642311096),
   ('Чеснок', 0.7399617433547974),
   ('перец', 0.5796153545379639)]),
 ('Тунец в соусе терияки',
  0.6937393546104431,
  [('Имбирь', 0.6141088008880615),
   ('Сахар', 0.6113570332527161),
   ('чеснок', 0.4926825761795044),
  

### Проблемы:

1.Модель находит похожесть по количеству общих слов.


Предолжения chat gpt:



1. Введение весовых коэффициентов для ингредиентов: Вы можете ввести веса для ингредиентов на основе их значимости в рецепте. Например, главный ингредиент блюда (такой как "курица" в куриных блюдах) может иметь более высокий вес, чем второстепенные ингредиенты (например, специи или приправы). Это может помочь модели учесть важность ингредиента в общем контексте блюда.

2. Использование расширенных методов семантического анализа: Вы можете использовать методы, которые учитывают семантическую близость слов, а не просто наличие или отсутствие слова в рецепте. Например, вместо использования простого векторного представления слов, можно использовать методы, основанные на контексте, такие как word2vec, GloVe или FastText. Эти методы позволяют определить близость между словами на основе их контекста в большом корпусе текстов, а не просто на основе их совместного появления в одном и том же рецепте.

3. Использование алгоритмов кластеризации: Вместо вычисления сходства между каждой парой рецептов, вы можете использовать алгоритмы кластеризации для группировки похожих рецептов. Это может быть более эффективным и масштабируемым подходом, особенно при работе с большими наборами данных.

4. Улучшение предобработки данных: Вы можете улучшить предобработку данных, чтобы уменьшить шум и увеличить точность модели. Например, вы можете убедиться, что все ингредиенты стандартизованы (например, "томаты" и "томат" должны быть одинаковы), удалить редкие ингредиенты или объединить похожие ингредиенты в одну категорию (например, "помидоры" и "черри").

5. Использование методов рекомендательных систем: Вы можете применить методырекомендательных систем, например, коллаборативную фильтрацию или факторизацию матриц, которые используются для рекомендации фильмов или музыки. Эти методы учитывают не только сходство между объектами (в данном случае рецептами), но и предпочтения пользователей.

6. Применение Doc2Vec вместо Word2Vec: Doc2Vec – это расширение Word2Vec, которое позволяет учитывать контекст всего документа (в данном случае, рецепта), а не только отдельных слов. Это может помочь улучшить точность сходства, поскольку оно учитывает общую структуру и контекст рецепта, а не только отдельные ингредиенты.

пробуем исключить ингридиент:

In [41]:
def find_similar_dishes(df, dish, exclude_ingredient,model):
    # Get the ingredients of the dish
    dish_ingredients = df.loc[df['meal'] == dish, 'ingredients'].values[0]

    # Exclude the ingredient
    dish_ingredients = [ingredient for ingredient in dish_ingredients if ingredient != exclude_ingredient]

    # If there are no ingredients left after excluding, return empty list
    if not dish_ingredients:
        return []

    # Calculate similarities
    similarities = df['ingredients'].apply(lambda x: recipe_similarity(dish_ingredients, x, model)) # Pass the model here

    # Exclude the original dish and dishes with the excluded ingredient
    similar_dishes = df[(df['meal'] != dish) & (~df['ingredients'].apply(lambda x: exclude_ingredient in x))]

    # Sort by similarity
    similar_dishes = similar_dishes.assign(similarity=similarities).sort_values(by='similarity', ascending=False)

    return similar_dishes


In [43]:
find_similar_dishes(df,'Шаурма с курицей','мясо',model)[['meal','category']]

,meal,category
20,Бургер классический,Фастфуд
9,Греческий,Салаты
8,Цезарь,Салаты
11,Стейк из свинины,Стейк
13,Тунец в соусе терияки,Рыба
4,Борщ,Супы
5,Солянка,Супы
3,Овсяная каша,Каша
0,Омлет с грибами,Омлеты
12,Лосось на гриле,Рыба


In [45]:
similar_words = model.most_similar('мясо', topn=20)
print(similar_words)

[('мяско', 0.7551317811012268), ('мясо.', 0.7505342960357666), ('Мясо', 0.7483969926834106), ('говядину', 0.7157896757125854), ('свинину', 0.7041599750518799), ('сало', 0.6996752619743347), ('курятину', 0.6891534924507141), ('мяса', 0.6782190799713135), ('мясцо', 0.662449836730957), ('молоко', 0.6586489677429199), ('курицу', 0.657137930393219), ('крольчатину', 0.6550291180610657), ('маринованное', 0.6516428589820862), ('жареное', 0.6498854756355286), ('телятину', 0.6440608501434326), ('свинина', 0.6413251757621765), ('поджаренное', 0.6401798129081726), ('жаренное', 0.6380771398544312), ('говядина', 0.6363006830215454), ('прожаренное', 0.6309633255004883)]


### ИТОГ.Возникшие проблемы:

1. модель сходства на `мясо` имеет высокую похожесть на `молоко` 0.64  но ниже на `свинина` 0.64. Это неправильно сработает для рекомендации похожих блюд на основе ингридиентов.

1. Получается что два слова, например `говяжий фарш`, система вообще не определяет.нужен вектор работающий с документами а не с отдельными словами возможно. Но какая это модель, или как перенастроить текущую?

2. Есть ли готовая языковая модель (в идеале для русского) которая покажет схожесть ингридиентов?


3. Какие самые рабочие подходы nlp для подобных задач?





